In [1]:
import torch

from haloflow.npe import valid as V
from haloflow.npe import plotting

cuda = torch.cuda.is_available()
device = ("cuda:0" if cuda else "cpu")

In [2]:
sims = ['Simba100', 'Eagle100', 'TNG50', 'TNG100', 'TNG_ALL'] 
obs = 'mags_morph_extra'

In [ ]:
train_sim = sims[0]
test_sim = sims[1]
ranks, alpha, ecp, _ = V.validate_npe(train_obs=obs, train_sim=train_sim, test_obs=obs, test_sim=test_sim, version=1)

917 models trained
[820, 330, 469, 747, 737]


/groups/chhahn/haloflow/src/haloflow/util.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  qphi = torch.load(fqphi, map_location=device)
/groups/chhahn/haloflow/src/halo

In [ ]:
fig = plotting.plot_rank_statistics([ranks], labels=[f'{train_sim}_{test_sim}'])

fig, ax = plotting.plot_coverage([alpha], [ecp], labels=[f'{train_sim}_{test_sim}'])